In [ ]:
%pip install pinecone

In [ ]:
%pip install sentence-transformers

In [3]:
from google.colab import userdata
PINECONE_API_KEY = userdata.get('PINECONE_API_KEY')

In [4]:
from pinecone import Pinecone

pc = Pinecone(api_key=PINECONE_API_KEY)
index = pc.Index("genai-test-index")

In [5]:
from google.colab import userdata
mongo_db_uri = userdata.get('mongo_db_uri')

In [ ]:
%pip install pymongo

In [7]:
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi

uri = mongo_db_uri

client = MongoClient(uri, server_api=ServerApi('1'))

try:
    client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    print(e)

Pinged your deployment. You successfully connected to MongoDB!


In [8]:
db = client['movie_db']

In [9]:
collection = db['collection02']

In [ ]:
from sentence_transformers import SentenceTransformer

embedding_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [ ]:
cursor = collection.watch(full_document='updateLookup')
print("Change Stream UP!")
while True:
    change = next(cursor)
    if change['operationType'] == 'insert':
      document = change['fullDocument']
      vector = embedding_model.encode(document['fullplot'])
      vector = vector.tolist()
      vector = [float(x) for x in vector]
      upsert_data = (str(document['_id']), vector)
      index.upsert([upsert_data])
    elif change['operationType'] == 'update':
      document = change['fullDocument']
      document_id = document['_id']
      updated_fields = change['updateDescription']['updatedFields']
      if updated_fields.get('fullplot'):
        vector = embedding_model.encode(updated_fields['fullplot'])
        upsert_data = (str(document_id), vector)
        index.upsert([upsert_data])
    elif change['operationType'] == 'delete':
      index.delete(ids=[str(change['documentKey']['_id'])])